**Modelo Sinapsia**

In [1]:
'''
Install:
- pandas
- numpy
- ssqueezepy
- mne
- scipy
- sklearn
- tensorflow
'''

'\nInstall:\n- pandas\n- numpy\n- ssqueezepy\n- mne\n- scipy\n- sklearn\n- tensorflow\n'

In [1]:
import pandas as pd
import glob
import ssqueezepy #para convertir la señal en imagen
from ssqueezepy import cwt
from ssqueezepy.visuals import plot, imshow
import os
import mne #eeg analysis library
import scipy.io
#import torch.nn as nn
#import torch
import numpy as np
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Ordenando el dataset (no correr)**

----------> Juntamos los archivos

In [ ]:
#subo el archivo con las ids y diagnósticos (y)
metadata = pd.read_csv('metadata_guineabissau.csv')
metadata.head(5)

In [ ]:
metadata = metadata.drop('Remarks', axis=1)
metadata = metadata.drop('recordedPeriod', axis=1)
metadata = metadata.drop('startTime', axis=1)
metadata = metadata.drop('Eyes.condition', axis=1)

metadata.head(5)

In [1]:
path = r'/content/eeg_data'
all_files = glob.glob(path + "/*.csv")

all_files = sorted(all_files)


def add_id_y (all_files, metadata):
  index = 0

  modified_dataframes_list = []
  non_eeg_columns = []

  for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    dataframe = df.iloc[:, 1:15].copy()
    dataframe['subject_id'] = metadata['subject.id'][index]
    dataframe['y'] = metadata['Group'][index]
    index += 1
    modified_dataframes_list.append(dataframe)

  return modified_dataframes_list

'''def join_csv (modified_dataframes_list):
  dataframes_list = []

  for filename in modified_dataframes_list:
    dataframe = pd.read_csv(filename, index_col=None, header=0)
    dataframes_list.append(dataframe)

  all_dataframes = pd.concat(dataframes_list, axis=0, ignore_index=True)
  all_dataframes.to_csv('eeg_data.csv', index=False)

  return all_dataframes'''

NameError: name 'glob' is not defined

In [ ]:
modified_dataframes = add_id_y(all_files, metadata)

dataframes_list = []

for dataframe in modified_dataframes:
  dataframes_list.append(dataframe)

all_dataframes = pd.concat(dataframes_list, axis=0, ignore_index=True)
all_dataframes.to_csv('eeg_data.csv', index=False)

In [ ]:
all_dataframes

**---------------Procesando el dataset para el modelo (formato mne)---------------**

**Colmna y**

Epilepsia == 1

Normal == 0

In [4]:
eeg_data = pd.read_csv('C:\\Users\\47575909\\Desktop\\eeg_data.csv')
eeg_data

,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8,subject_id,y
0,4426.153846,3994.871795,4408.205128,3847.692308,4690.256410,3895.897436,4702.051282,3914.871795,4049.743590,4297.948718,4186.153846,4132.820513,4106.666667,4076.410256,1,Epilepsy
1,4420.512821,3986.666667,4394.358974,3836.923077,4678.461538,3886.666667,4696.410256,3910.769231,4054.358974,4295.897436,4187.179487,4126.666667,4105.641026,4065.128205,1,Epilepsy
2,4413.846154,3986.153846,4386.666667,3831.794872,4654.871795,3881.025641,4690.769231,3908.205128,4066.666667,4301.025641,4188.205128,4123.076923,4103.076923,4063.589744,1,Epilepsy
3,4407.692308,3984.615385,4384.102564,3832.820513,4644.615385,3883.076923,4686.153846,3910.256410,4063.076923,4298.974359,4184.615385,4127.179487,4095.384615,4071.282051,1,Epilepsy
4,4407.179487,3978.974359,4382.564103,3832.307692,4647.692308,3878.974359,4685.641026,3903.076923,4057.948718,4295.384615,4182.564103,4128.205128,4090.769231,4064.102564,1,Epilepsy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3850363,4211.794872,3862.051282,4139.487179,3670.256410,4327.179487,3662.564103,4557.435897,3894.358974,4238.974359,4189.743590,4010.256410,4126.153846,3830.769231,3883.589744,97,Control
3850364,4209.743590,3861.538462,4138.974359,3667.179487,4323.589744,3662.051282,4556.410256,3893.846154,4232.820513,4188.205128,4005.641026,4125.128205,3829.230769,3878.974359,97,Control
3850365,4211.282051,3862.564103,4136.410256,3671.282051,4324.102564,3664.102564,4556.923077,3896.923077,4235.384615,4185.128205,4005.128205,4123.076923,3829.230769,3882.051282,97,Control
3850366,4217.435897,3866.666667,4140.000000,3676.923077,4326.153846,3668.205128,4558.974359,3901.025641,4234.871795,4184.615385,4010.256410,4122.564103,3829.230769,3890.769231,97,Control


In [5]:
eeg_data = eeg_data.round(3)
eeg_data

,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8,subject_id,y
0,4426.154,3994.872,4408.205,3847.692,4690.256,3895.897,4702.051,3914.872,4049.744,4297.949,4186.154,4132.821,4106.667,4076.410,1,Epilepsy
1,4420.513,3986.667,4394.359,3836.923,4678.462,3886.667,4696.410,3910.769,4054.359,4295.897,4187.179,4126.667,4105.641,4065.128,1,Epilepsy
2,4413.846,3986.154,4386.667,3831.795,4654.872,3881.026,4690.769,3908.205,4066.667,4301.026,4188.205,4123.077,4103.077,4063.590,1,Epilepsy
3,4407.692,3984.615,4384.103,3832.821,4644.615,3883.077,4686.154,3910.256,4063.077,4298.974,4184.615,4127.179,4095.385,4071.282,1,Epilepsy
4,4407.179,3978.974,4382.564,3832.308,4647.692,3878.974,4685.641,3903.077,4057.949,4295.385,4182.564,4128.205,4090.769,4064.103,1,Epilepsy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3850363,4211.795,3862.051,4139.487,3670.256,4327.179,3662.564,4557.436,3894.359,4238.974,4189.744,4010.256,4126.154,3830.769,3883.590,97,Control
3850364,4209.744,3861.538,4138.974,3667.179,4323.590,3662.051,4556.410,3893.846,4232.821,4188.205,4005.641,4125.128,3829.231,3878.974,97,Control
3850365,4211.282,3862.564,4136.410,3671.282,4324.103,3664.103,4556.923,3896.923,4235.385,4185.128,4005.128,4123.077,3829.231,3882.051,97,Control
3850366,4217.436,3866.667,4140.000,3676.923,4326.154,3668.205,4558.974,3901.026,4234.872,4184.615,4010.256,4122.564,3829.231,3890.769,97,Control


In [6]:
#paso los valores categóricos a numéricos
mapping = {'Epilepsy': 1, 'Control': 0}
eeg_data['y'] = eeg_data['y'].map(mapping)
eeg_data

,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8,subject_id,y
0,4426.154,3994.872,4408.205,3847.692,4690.256,3895.897,4702.051,3914.872,4049.744,4297.949,4186.154,4132.821,4106.667,4076.410,1,1
1,4420.513,3986.667,4394.359,3836.923,4678.462,3886.667,4696.410,3910.769,4054.359,4295.897,4187.179,4126.667,4105.641,4065.128,1,1
2,4413.846,3986.154,4386.667,3831.795,4654.872,3881.026,4690.769,3908.205,4066.667,4301.026,4188.205,4123.077,4103.077,4063.590,1,1
3,4407.692,3984.615,4384.103,3832.821,4644.615,3883.077,4686.154,3910.256,4063.077,4298.974,4184.615,4127.179,4095.385,4071.282,1,1
4,4407.179,3978.974,4382.564,3832.308,4647.692,3878.974,4685.641,3903.077,4057.949,4295.385,4182.564,4128.205,4090.769,4064.103,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3850363,4211.795,3862.051,4139.487,3670.256,4327.179,3662.564,4557.436,3894.359,4238.974,4189.744,4010.256,4126.154,3830.769,3883.590,97,0
3850364,4209.744,3861.538,4138.974,3667.179,4323.590,3662.051,4556.410,3893.846,4232.821,4188.205,4005.641,4125.128,3829.231,3878.974,97,0
3850365,4211.282,3862.564,4136.410,3671.282,4324.103,3664.103,4556.923,3896.923,4235.385,4185.128,4005.128,4123.077,3829.231,3882.051,97,0
3850366,4217.436,3866.667,4140.000,3676.923,4326.154,3668.205,4558.974,3901.026,4234.872,4184.615,4010.256,4122.564,3829.231,3890.769,97,0


In [7]:
#función para convertir los datos a formato mne
def convertDF2MNE(sub):
    info = mne.create_info(list(sub.columns), ch_types=['eeg'] * len(sub.columns), sfreq=128)
    info.set_montage('standard_1020')
    data=mne.io.RawArray(sub.T, info)
    data.set_eeg_reference()
    #data.filter(l_freq=0.1,h_freq=45)
    epochs = mne.make_fixed_length_epochs(data, duration = 3, overlap = 2)
    return epochs.get_data()

In [8]:
#agrupo las líneas que tienen el mismo id, o sea que son del mismo eeg
groups = eeg_data.groupby('subject_id')

#hago una lista con cada id de cada subject (hay 97 eeg)
grp_keys = list(groups.groups.keys())
print(grp_keys)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97]


In [9]:
from tqdm import tqdm

In [10]:
#listas para almacenar los datos
data_list = []

for i, grpno in tqdm(enumerate(grp_keys)):
    grp = groups.get_group(grpno).drop(['subject_id'], axis=1)
    label = int(grp['y'].unique())
    subject_id = grpno
    grp = grp.drop('y', axis=1)

    #convierto mi info a formato mne
    data = convertDF2MNE(grp)  #(trials, channels, length)

    for c, x in enumerate(data):  #loop trials
        Wx, scales = cwt(x, 'morlet')
        Wx = np.abs(Wx)
        path = os.path.join('C:\\Users\\47575909\\Desktop\\scaleogram', f'subvideo_{grpno}/')
        os.makedirs(path, exist_ok=True)
        path = os.path.join(path, f'trial_{c}.npy')  # Corrección en la construcción de la ruta
        np.save(path, Wx)

        data_entry = {
            'grpno': i,
            'label': label,
            'path': path
        }

        #guardo la info del grupo (en realidad sería el "id" del eeg), la label y la path del archivo literal del eeg que ya se procesó
        data_list.append(data_entry)

#creo un dataset de TensorFlow
dataset = tf.data.Dataset.from_generator(
    generator=lambda: (data_entry for data_entry in data_list),
    output_signature={
        'grpno': tf.TensorSpec(shape=(), dtype=tf.int32),
        'label': tf.TensorSpec(shape=(), dtype=tf.int32),
        'path': tf.TensorSpec(shape=(), dtype=tf.string)
    }
)

0it [00:00, ?it/s]

Creating RawArray with float64 data, n_channels=14, n_times=38528
    Range : 0 ... 38527 =      0.000 ...   300.992 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


C:\Users\47575909\AppData\Local\Temp\ipykernel_6176\2171158857.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(grp['y'].unique())


Not setting metadata
299 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 299 events and 384 original time points ...
0 bad epochs dropped


1it [00:17, 17.36s/it]

...

C:\Users\47575909\AppData\Local\Temp\ipykernel_6176\2171158857.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(grp['y'].unique())
95it [26:21, 16.18s/it]

Creating RawArray with float64 data, n_channels=14, n_times=37632
    Range : 0 ... 37631 =      0.000 ...   293.992 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Not setting metadata
292 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 292 events and 384 original time points ...
0 bad epochs dropped


C:\Users\47575909\AppData\Local\Temp\ipykernel_6176\2171158857.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(grp['y'].unique())
96it [26:37, 16.01s/it]

Creating RawArray with float64 data, n_channels=14, n_times=38400
    Range : 0 ... 38399 =      0.000 ...   299.992 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Not setting metadata
298 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 298 events and 384 original time points ...
0 bad epochs dropped


C:\Users\47575909\AppData\Local\Temp\ipykernel_6176\2171158857.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(grp['y'].unique())
97it [26:53, 16.63s/it]


In [11]:
#función para serializar un ejemplo en un TFRecord
def serialize(grpno, label, path):
    feature = {
        'grpno': tf.train.Feature(int64_list=tf.train.Int64List(value=[grpno])),
        'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
        'path': tf.train.Feature(bytes_list=tf.train.BytesList(value=[path.encode('utf-8')])),
    }
    
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    
    return example_proto.SerializeToString()

In [12]:
TFdataset = './tfrecord_data/'
os.makedirs(TFdataset)

#función para guardar los datos de cada grupo
def save_group_to_tfrecord(grpno, dataset):
    group_file_dataset = os.path.join(TFdataset, f'{grpno}_data.tfrecord')
    
    with tf.io.TFRecordWriter(group_file_dataset) as writer:
        for data_entry in dataset:
            grpno = data_entry['grpno'].numpy()
            label = data_entry['label'].numpy()
            path = data_entry['path'].numpy().decode('utf-8')
            serialized = serialize(grpno, label, path)
            writer.write(serialized)
    
    print(f'Datos del grupo {grpno} se guardó en {group_file_dataset}')

In [13]:
for grpno in grp_keys:
    group_dataset = dataset.filter(lambda x: x['grpno'] == grpno)
    save_group_to_tfrecord(grpno, group_dataset)

Datos del grupo 1 se guardó en ./tfrecord_data/1_data.tfrecord
Datos del grupo 2 se guardó en ./tfrecord_data/2_data.tfrecord
Datos del grupo 3 se guardó en ./tfrecord_data/3_data.tfrecord
Datos del grupo 4 se guardó en ./tfrecord_data/4_data.tfrecord
Datos del grupo 5 se guardó en ./tfrecord_data/5_data.tfrecord
Datos del grupo 6 se guardó en ./tfrecord_data/6_data.tfrecord
Datos del grupo 7 se guardó en ./tfrecord_data/7_data.tfrecord
Datos del grupo 8 se guardó en ./tfrecord_data/8_data.tfrecord
Datos del grupo 9 se guardó en ./tfrecord_data/9_data.tfrecord
Datos del grupo 10 se guardó en ./tfrecord_data/10_data.tfrecord
Datos del grupo 11 se guardó en ./tfrecord_data/11_data.tfrecord
Datos del grupo 12 se guardó en ./tfrecord_data/12_data.tfrecord
Datos del grupo 13 se guardó en ./tfrecord_data/13_data.tfrecord
Datos del grupo 14 se guardó en ./tfrecord_data/14_data.tfrecord
Datos del grupo 15 se guardó en ./tfrecord_data/15_data.tfrecord
Datos del grupo 16 se guardó en ./tfrecord_

In [4]:
#ACA SUBO LOS ARCHIVOS DE TFRECORD ||||||||||||||||||||||||||||||||||||||||||||
TFrecord_directory = 'C:\\Users\\47575909\\Desktop\\tfrecord_data' 

#lista de archivos TFRecord en la carpeta
filenames = [os.path.join(TFrecord_directory, f) for f in os.listdir(TFrecord_directory) if f.endswith('.tfrecord')]

#Hago un dataset de TensorFlow a partir de los archivos TFRecord
dataset = tf.data.TFRecordDataset(filenames)

#función para parsear los datos de TFRecord
def parse_tfrecord_fn(example):
    feature_description = {
        'grpno': tf.io.FixedLenFeature([], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.int64),
        'path': tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, feature_description)
    return example

parsed_dataset = dataset.map(parse_tfrecord_fn)

#batching para argar los datos en lotes
batch_size = 5  #puedo cambiar el tamaño (en la PC del colegio el máximo es 16, pero capáz que le tengo que poner menos)
batched_dataset = parsed_dataset.batch(batch_size)

In [2]:
#ACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
#PRUEBO CON MENOS DATOS
#ACA SUBO LOS ARCHIVOS DE TFRECORD
TFrecord_directory = 'C:\\Users\\47575909\\Desktop\\tfrecord_data_prueba' 

#lista de archivos TFRecord en la carpeta
filenames = [os.path.join(TFrecord_directory, f) for f in os.listdir(TFrecord_directory) if f.endswith('.tfrecord')]

#Hago un dataset de TensorFlow a partir de los archivos TFRecord
dataset = tf.data.TFRecordDataset(filenames)

#función para parsear los datos de TFRecord
def parse_tfrecord_fn(example):
    feature_description = {
        'grpno': tf.io.FixedLenFeature([], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.int64),
        'path': tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, feature_description)
    return example

parsed_dataset = dataset.map(parse_tfrecord_fn)

#batching para argar los datos en lotes
batch_size = 5  #puedo cambiar el tamaño (en la PC del colegio el máximo es 16, pero capáz que le tengo que poner menos)
batched_dataset = parsed_dataset.batch(batch_size)

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'C:\\Users\\47575909\\Desktop\\tfrecord_data_prueba'

#### Modelo (CNN) 

In [6]:
#SEPARO EN X E Y
X = []
y = []

#itero los lotes
for batch_data in tqdm(batched_dataset):
    paths = batch_data['path'].numpy()
    labels = batch_data['label'].numpy()
    
    #cargo los scaleograms desde los archivos .npy
    batch_X = []
    for path in paths:
        spectrogram = np.load(path.decode('utf-8'))
        batch_X.append(spectrogram)
    
    #agrego los scaleograms y labels a las listas X y y
    X.extend(batch_X)
    y.extend(labels)

#convierto X e y a formato .npy
X = np.array(X)
y = np.array(y)

print('Ya se separó todo!')
print('Salió todo bien, tranquina :))')

2526it [03:35, 11.74it/s]


MemoryError: Unable to allocate 4.72 MiB for an array with shape (1236480,) and data type float32

In [6]:
X.shape

(606, 14, 230, 384)

In [11]:
# Aplanar tus datos
X = X.reshape(606, -1)

In [4]:
#NORMALIZO X en rango [-1, 1] para que el cero quede en el centro
min_val = np.min(X)
max_val = np.max(X)

X = -1 + 2 * (X - min_val) / (max_val - min_val)

#X = (X - X.min()) / (X.max() - X.min()) [0, 1]

In [13]:
X.shape

(606, 1236480)

In [5]:
#LOS GUARDO EN MI COMPUTADORA
#np.save('X.npy', X) está guardada normalizada en rango [0, 1], habría que cambiar eso
np.save('X.npy', X) #esto es para probar
np.save('y.npy', y)

In [2]:
# Cargar los arreglos X e y desde los archivos .npy (PARA CUANDO EL CÓDIGO ANTERIOR YA ESTÁ CORRIDO Y NO QUIERO VOLVERLO A CORRER :d)
#X = np.load('C:\\Users\\47575909\\Desktop\\X_1.npy')
X = np.load('C:\\Users\\47575909\\Desktop\\X_PCA.npy')
y = np.load('C:\\Users\\47575909\\Desktop\\y_PCA.npy')

In [3]:
shape = X.shape

In [4]:
shape

(29588, 1000)

In [5]:
#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

#Divido los datos de test de los de evaluation (50/50 del 30% anterior)
X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)

cwt.shape = (14, 230, 384)

- Alto: 230 (número de filas en tus imágenes CWT).
- Ancho: 384 (número de columnas en tus imágenes CWT).
- Canales: 14 (número de canales EEG).

In [6]:
#si es que me toma mal la imput shape seguramente es por la longitud de la grabación de cada eeg, 
# por lo que dice la info del dataset no todos los eeg tienen la exacta misma duración

In [7]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)

In [8]:
X_train.shape

(20711, 1000, 1)

In [9]:
#CREO LA CNN
# Definir una capa de entrada compatible con los datos después de PCA
#input_layer = tf.keras.layers.Input(shape=(1000,))

from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Resto de tu modelo
model = tf.keras.models.Sequential([
    Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(1000, 1)), #le agrego un canal imaginario para que me deje la input shape
    MaxPooling1D(pool_size=2),
    
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),

    Flatten(),
    
    Dense(32, activation='relu'),
    Dropout(0.5),
    
    Dense(1, activation='sigmoid')
])

'''model = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape = (14, 230, 384)), 
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(32, activation='relu'),

    tf.keras.layers.Dense(1, activation='sigmoid') #función sigmoide porque yo quiero que me devuelva un 1 de "si epilepsia" o un 0 de "no epilepsia"
])'''

#(LA IMPUT SHAPE LA SAQUÉ CON LO DE Wx, AGARRÉ UN EJEMPLO DE UNA PRESONA Y ME FIJÉ EL TAMAÑO PORQUE YO LE VOY A DAR INFO DE PACIENTES DE ESE FORMATO A LA CNN)

'model = tf.keras.models.Sequential([\n    \n    tf.keras.layers.Conv2D(128, (3,3), activation=\'relu\', input_shape = (14, 230, 384)), \n    tf.keras.layers.MaxPooling2D(2, 2),\n\n    tf.keras.layers.Conv2D(64, (3,3), activation=\'relu\'),\n    tf.keras.layers.MaxPooling2D(2,2),\n\n    tf.keras.layers.Dropout(0.5),\n    tf.keras.layers.Flatten(),\n\n    tf.keras.layers.Dense(32, activation=\'relu\'),\n\n    tf.keras.layers.Dense(1, activation=\'sigmoid\') #función sigmoide porque yo quiero que me devuelva un 1 de "si epilepsia" o un 0 de "no epilepsia"\n])'

In [10]:
#COMPILO
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [11]:
#ENTRENO
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
test_loss, test_acc = model.evaluate(X_val, y_val)

#capaz que debería probar con cross validation después

#ME DA PARA EL ORTO, PERO INTENTAR CON TECNICAS DE AUMENTO DE DATOS Y AGREANDAR EL DATASET D:

Epoch 1/10
648/648 [==============================] - 39s 59ms/step - loss: 0.6929 - accuracy: 0.5224 - val_loss: 0.6900 - val_accuracy: 0.5410
Epoch 2/10
648/648 [==============================] - 40s 61ms/step - loss: 0.6911 - accuracy: 0.5281 - val_loss: 0.6905 - val_accuracy: 0.5225
Epoch 3/10
648/648 [==============================] - 40s 62ms/step - loss: 0.6910 - accuracy: 0.5315 - val_loss: 0.6874 - val_accuracy: 0.5539
Epoch 4/10
648/648 [==============================] - 40s 61ms/step - loss: 0.6902 - accuracy: 0.5345 - val_loss: 0.6881 - val_accuracy: 0.5543
Epoch 5/10
648/648 [==============================] - 39s 60ms/step - loss: 0.6895 - accuracy: 0.5381 - val_loss: 0.6882 - val_accuracy: 0.5424
Epoch 6/10
648/648 [==============================] - 40s 62ms/step - loss: 0.6899 - accuracy: 0.5397 - val_loss: 0.6877 - val_accuracy: 0.5417
Epoch 7/10
648/648 [==============================] - 39s 60ms/step - loss: 0.6891 - accuracy: 0.5400 - val_loss: 0.6864 - val_accuracy:

In [12]:
#VEO LA PERFORMANCE
print("Accuracy : ", test_acc)
print("Loss : ", test_loss)

Accuracy :  0.5538530945777893
Loss :  0.687100887298584


In [13]:
max_shift = 50  # Máximo desplazamiento en muestras
X_augmented = []
y_augmented = []

for sample, label in zip(X_train, y_train):
    shift = np.random.randint(-max_shift, max_shift)
    
    augmented_sample = np.roll(sample, shift)
    X_augmented.append(augmented_sample)
    
    augmented_label = np.roll(label, -shift)  # Alinea las etiquetas con el desplazamiento
    y_augmented.append(augmented_label)

X_augmented = np.array(X_augmented)
y_augmented = np.array(y_augmented)

In [14]:
#datos generadossssssssssss
model.fit(X_augmented, y_augmented, epochs=10, validation_data=(X_val, y_val))

test_loss, test_acc = model.evaluate(X_val, y_val)

Epoch 1/10
648/648 [==============================] - 39s 61ms/step - loss: 0.6943 - accuracy: 0.5161 - val_loss: 0.6921 - val_accuracy: 0.5212
Epoch 2/10
648/648 [==============================] - 39s 60ms/step - loss: 0.6925 - accuracy: 0.5185 - val_loss: 0.6901 - val_accuracy: 0.5408
Epoch 3/10
648/648 [==============================] - 38s 59ms/step - loss: 0.6924 - accuracy: 0.5192 - val_loss: 0.6918 - val_accuracy: 0.5237
Epoch 4/10
648/648 [==============================] - 38s 59ms/step - loss: 0.6926 - accuracy: 0.5218 - val_loss: 0.6914 - val_accuracy: 0.5300
Epoch 5/10
648/648 [==============================] - 38s 59ms/step - loss: 0.6920 - accuracy: 0.5258 - val_loss: 0.6895 - val_accuracy: 0.5480
Epoch 6/10
648/648 [==============================] - 38s 59ms/step - loss: 0.6919 - accuracy: 0.5249 - val_loss: 0.6894 - val_accuracy: 0.5475
Epoch 7/10
648/648 [==============================] - 38s 59ms/step - loss: 0.6918 - accuracy: 0.5277 - val_loss: 0.6906 - val_accuracy:

In [15]:
print("Accuracy : ", test_acc)
print("Loss : ", test_loss)

Accuracy :  0.5277152061462402
Loss :  0.6904980540275574


# ACÁ EXPORTAMOS EL MODELO

In [ ]:
#guardamos el modelo
model.save('modelo_SinapsIA.h5')

Álvaro, hola, yo te diría que empecemos con el modelo casi sin entrenar para que ya sepamos como pasarlo a la web. Vos lo que vas a hacer es pedirme los archivos para entrenarlo poquito (que literalmente van a ser solo 2 eeg porque sino va a ser muy pesado, yo estoy trabajando en reducir el tamaño del dataset). Entonces, vos te tendrías que descargar este notebook y correrlo en tu compu y entrenarlo así con estos pocos datos que no te van a pesar (se supone), y ahí lo conectas con lo de Jano. Para cuando los datos ya estén bien, supongo que tendremos que hacer lo mismo pero desde la pc porque aunque le reduzca el tamaño va a ser imposible que entre en la netbook. Solamente hay que ejecutar el código que vos hagas con el mismo modelo pero entrenado con más datos.

ADEMÁS, TE HICE UN NOTEBOOK CON TODO PASADO, PERO IUAL MIRA ESTE.

In [ ]:
#investigar PCA para tensores (TAMBIÉN INVESTGAR MÁS PARA TENSORES)
#https://youtu.be/FgakZw6K1QQ?si=GrRc8tCp2eEfNKaX

#TAMBIÉN CHONA ME RECOMIENDA UNA LIBRERÍA PaCMACP para mi caso que uso tensores de tensorflow

**Tengo que hacer y notas**
##### **--> pasar todo a una pc**
##### **--> separar en datos de train, test y validation**

`78` = train

`10` = validation

`9` = test

##### **--> ver el formato y como feedearselo a una red neuronal**

Opciones de modelos a investigar:

*   `Convolutional neural network (CNN)`
*   `Recurrent neural network (RNN)`

